In [4]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [5]:
adeft_grounder = AdeftGrounder()

In [6]:
shortforms = ['IBD']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [60]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [61]:
list(zip(longforms, counts))

[('inflammatory bowel disease', 3080),
 ('integrase binding domain', 33),
 ('identical by descent', 31),
 ('in binding domain', 19),
 ('infectious bursal disease', 16),
 ('isolation by distance', 15),
 ('inﬂammatory bowel disease', 12),
 ('inflammatory bowel disorders', 6),
 ('immune benefit disabled', 4),
 ('irif binding domain', 4),
 ('intrabony defect', 4),
 ('iliac branch devices', 2),
 ('intra bony defect', 2),
 ('inherited bleeding disorders', 2),
 ('indentity by descent', 2),
 ('inflammation bowel disease', 2),
 ('inhaled bronchodilators', 2),
 ('inflammatory bowel disease group', 2),
 ('isobutyryl coa dehydrogenase', 2)]

In [65]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [66]:
result = [grounding_map, names, pos_labels]

In [67]:
result

[{'identical by descent': 'ungrounded',
  'iliac branch devices': 'ungrounded',
  'immune benefit disabled': 'ungrounded',
  'in binding domain': 'ungrounded',
  'indentity by descent': 'ungrounded',
  'infectious bursal disease': 'MESH:D007243',
  'inflammation bowel disease': 'NCIT:C157711',
  'inflammatory bowel disease': 'NCIT:C157711',
  'inflammatory bowel disease group': 'NCIT:C157711',
  'inflammatory bowel disorders': 'NCIT:C157711',
  'inhaled bronchodilators': 'ungrounded',
  'inherited bleeding disorders': 'ungrounded',
  'integrase binding domain': 'IP:IPR021567',
  'intra bony defect': 'ungrounded',
  'intrabony defect': 'ungrounded',
  'inﬂammatory bowel disease': 'NCIT:C157711',
  'irif binding domain': 'ungrounded',
  'isobutyryl coa dehydrogenase': 'HGNC:87',
  'isolation by distance': 'ungrounded'},
 {'MESH:D007243': 'Infectious bursal disease virus',
  'NCIT:C157711': 'Inflammatory Bowel Disease',
  'IP:IPR021567': 'Lens epithelium-derived growth factor, integrase-b

In [68]:
grounding_map, names, pos_labels = [{'identical by descent': 'ungrounded',
  'iliac branch devices': 'ungrounded',
  'immune benefit disabled': 'ungrounded',
  'in binding domain': 'ungrounded',
  'indentity by descent': 'ungrounded',
  'infectious bursal disease': 'MESH:D007243',
  'inflammation bowel disease': 'NCIT:C157711',
  'inflammatory bowel disease': 'NCIT:C157711',
  'inflammatory bowel disease group': 'NCIT:C157711',
  'inflammatory bowel disorders': 'NCIT:C157711',
  'inhaled bronchodilators': 'ungrounded',
  'inherited bleeding disorders': 'ungrounded',
  'integrase binding domain': 'IP:IPR021567',
  'intra bony defect': 'ungrounded',
  'intrabony defect': 'ungrounded',
  'inﬂammatory bowel disease': 'NCIT:C157711',
  'irif binding domain': 'ungrounded',
  'isobutyryl coa dehydrogenase': 'HGNC:87',
  'isolation by distance': 'ungrounded'},
 {'MESH:D007243': 'Infectious bursal disease virus',
  'NCIT:C157711': 'Inflammatory Bowel Disease',
  'IP:IPR021567': 'Lens epithelium-derived growth factor, integrase-binding domain',
  'HGNC:87': 'ACAD8'},
 ['HGNC:87', 'IP:IPR021567', 'MESH:D007243', 'NCIT:C157711']]

In [69]:
excluded_longforms = ['in binding domain']

In [70]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [71]:
additional_entities = {'HGNC:87': ['ACAD8', ['IBD', 'Isobutyryl', 'isobutyryl', 'IBDH']]}

In [72]:
unambiguous_agent_texts = {}

In [73]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [74]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [75]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [76]:
intersection1

[('HGNC:87', 'HGNC:87', 23)]

In [77]:
intersection2

[('MESH:D007243', 'HGNC:87', 0),
 ('NCIT:C157711', 'HGNC:87', 0),
 ('HGNC:87', 'HGNC:87', 1),
 ('ungrounded', 'HGNC:87', 0),
 ('IP:IPR021567', 'HGNC:87', 0)]

In [78]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [79]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [80]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 01:15:23] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 01:15:55] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9939795730265766 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [81]:
classifier.stats

{'label_distribution': {'MESH:D007243': 14,
  'NCIT:C157711': 2150,
  'HGNC:87': 6,
  'ungrounded': 40,
  'IP:IPR021567': 27},
 'f1': {'mean': 0.99398, 'std': 0.002377},
 'precision': {'mean': 0.99188, 'std': 0.002282},
 'recall': {'mean': 0.997269, 'std': 0.001702},
 'NCIT:C157711': {'f1': {'mean': 0.995833, 'std': 0.001176},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.991703, 'std': 0.002334}},
 'ungrounded': {'f1': {'mean': 0.821978, 'std': 0.043068},
  'pr': {'mean': 0.7, 'std': 0.061237},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D007243': {'f1': {'mean': 0.693333, 'std': 0.190205},
  'pr': {'mean': 0.566667, 'std': 0.249444},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'IP:IPR021567': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:87': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [82]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [83]:
disamb.dump(model_name, results_path)

In [84]:
print(disamb.info())

Disambiguation model for IBD

Produces the disambiguations:
	ACAD8*	HGNC:87
	Infectious bursal disease virus*	MESH:D007243
	Inflammatory Bowel Disease*	NCIT:C157711
	Lens epithelium-derived growth factor, integrase-binding domain*	IP:IPR021567

Class level metrics:
--------------------
Grounding                                                      	Count	F1     
                                     Inflammatory Bowel Disease*	2150	0.99583
                                                     Ungrounded	  40	0.82198
Lens epithelium-derived growth factor, integrase-binding domain*	  27	    1.0
                                Infectious bursal disease virus*	  14	0.69333
                                                          ACAD8*	   6	    1.0

Weighted Metrics:
-----------------
	F1 score:	0.99398
	Precision:	0.99188
	Recall:		0.99727

* Positive labels
See Docstring for explanation



In [89]:
model_to_s3(disamb)